In [128]:
from sqlalchemy import create_engine
from collections import OrderedDict
from datetime import datetime 
from typing import Union
from numpy import NaN
import pandas as pd
import numpy as np
import psycopg2
import urllib
import shutil
import os


In [129]:
db_setting = pd.read_csv(os.getcwd()+'\\'+'set_file'+'\\'+'auto_setting'+'\\'+'cwb_auto_settting.csv',encoding='ANSI') #database設定檔
hydro_setting = pd.read_csv(f'{os.getcwd()}'+'\\'+'set_file'+'\\'+'auto_setting'+'\\'+'cwb_auto_item.csv',encoding='ANSI') #讀取氣象水文設定檔
station_setting = pd.read_csv(f'{os.getcwd()}'+'\\'+'set_file'+'\\'+'auto_setting'+'\\'+'cwb_auto_station.csv',encoding='ANSI') #讀取氣象水文設定檔

In [130]:
#get db_setting
year_from = int(f'{db_setting["Input"][0]}')
year_to = int(f'{db_setting["Input"][1]}')
database = f'{db_setting["Input"][2]}'
user = f'{db_setting["Input"][3]}'
password = f'{db_setting["Input"][4]}'
host = f'{db_setting["Input"][5]}'
port = f'{db_setting["Input"][6]}'
database_table = f'{db_setting["Input"][7]}'
gssencmode = f'{db_setting["Input"][8]}'
output_data = f'{db_setting["Input"][9]}'

#get hydro_setting
data_input = []
hydro_setting.dropna(subset=["Input"], inplace=True)
for i in hydro_setting["Input"]:  
    data_input.append(i)
if len(data_input) == 1:
    column_list = data_input[0]
else:
    column_list = ", ".join(data_input)
    
#get station_setting
sta_input = []
station_setting.dropna(subset=["Input(station_no)"], inplace=True)
for i in station_setting["Input(station_no)"]:  
    sta_input.append(i)
if len(sta_input) == 1:
    sta_list = sta_input[0]
    sta_list = [i for i in set(sta_input)]
    sta_list = tuple(sta_list)
else:
    sta_list = [i for i in set(sta_input)]
    sta_list = tuple(sta_list)


In [131]:
#連接資料庫
engine = create_engine(f'postgresql+psycopg2://{user}:%s@{host}/{database}?gssencmode={gssencmode}'%urllib.parse.quote_plus(password))
#讀取資料庫
conn = psycopg2.connect(database=database,
                        user=user,
                        password=password,
                        host=host,
                        port=port,
                        gssencmode=gssencmode)


In [132]:
# read what methods does the usr want to use on each input data (1:min / 2:max / 3: mean / 4: sum)
input_methods ={}
for item,method_num in zip(hydro_setting['Input'],hydro_setting['method(1:min, max / 2min / 3: max / 4: mean/ 5. sum)']): 
    input_methods[item] = method_num

In [152]:
def cal_variable(var_name: str, calculate_method: str) -> pd.DataFrame:
    if var_name in input_methods:
        if str(input_methods[var_name]) == '5':
            all_df[var_name] = df.groupby('stno').resample('D').sum()[var_name].values
            all_df[var_name] = all_df[var_name].apply(lambda x:round(x,2))
        elif str(input_methods[var_name]) == '4':
            all_df[var_name] = df.groupby('stno').resample('D').mean()[var_name].values
            all_df[var_name] = all_df[var_name].apply(lambda x:round(x,2))
        elif str(input_methods[var_name]) == '3':
            all_df[var_name+"_max"] = df.groupby('stno').resample('D').max()[var_name].values
            all_df[var_name] = all_df[var_name].apply(lambda x:round(x,2))
        elif str(input_methods[var_name]) == '2':
            all_df[var_name+"_min"] = df.groupby('stno').resample('D').min()[var_name].values
            all_df[var_name] = all_df[var_name].apply(lambda x:round(x,2))
        elif str(input_methods[var_name]) == '1':
            all_df[var_name+"_min"] = df.groupby('stno').resample('D').min()[var_name].values
            all_df[var_name+"_min"] = all_df[var_name+"_min"].apply(lambda x:round(x,2))
            all_df[var_name+"_max"] = df.groupby('stno').resample('D').max()[var_name].values
            all_df[var_name+"_max"] = all_df[var_name+"_max"].apply(lambda x:round(x,2))
            
def ten_days_cal(df:pd.DataFrame(),var:str)->pd.DataFrame():
    if input_methods[var] == 5:
        df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].sum(),2)
    if input_methods[var] == 4:
        df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].mean(),2)
    if input_methods[var] == 3:
        df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].max(),2)
    if input_methods[var] == 2:
        df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].min(),2)
    if input_methods[var] == 1:
        df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].min(),2)
        df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].max(),2)

In [178]:
sta_list

('C0G650', 'C1X090')

In [ ]:
auto_final_df = pd.DataFrame()

for i in range(int(year_from),int(year_to)+1):
    if db_setting['Input'][9] =='1':
        query = f'select obsdate, obshour,stno, {column_list} from "{database_table}_{i}" where stno in {sta_list}'
        df = pd.read_sql(query,con=engine)
        df.reset_index(drop=True,inplace=True)
        auto_final_df = pd.concat([auto_final_df,df])

    if db_setting['Input'][9] =='2':
        query = f'select obsdate, obshour,stno, {column_list} from "{database_table}_{i}" where stno in {sta_list}'
        df = pd.read_sql(query,con=engine)
        df['obstime'] = pd.to_datetime(df['obsdate'].astype(str)+'_'+df['obshour'].astype(str),format='%Y-%m-%d_%H')
        df[df.isin([-9999,-9997,-9996,-9991])] = np.nan
        df.set_index(df['obstime'],inplace=True)
        all_df = pd.DataFrame()
        for j in input_methods:
            cal_variable(f'{j}', f"{input_methods[j]}")
        all_df['obstime'] = df.groupby('stno').resample('D')['obstime'].first().values
        all_df['stno'] = df.groupby('stno').resample('D')['stno'].first().values
        all_dff = all_df.copy()
        auto_final_df = pd.concat([auto_final_df,all_dff])
     
    if db_setting['Input'][9] =='3':
        query = f'select obsdate, obshour,stno, {column_list} from "{database_table}_{i}" where stno in {sta_list}'
        df = pd.read_sql(query,con=engine)
        df['obstime'] = pd.to_datetime(df['obsdate'].astype(str)+'_'+df['obshour'].astype(str),format='%Y-%m-%d_%H')
        df[df.isin([-9999,-9997,-9996,-9991])] = np.nan
        df.set_index(df['obstime'],inplace=True)
        all_df = pd.DataFrame()
        for j in input_methods:
            cal_variable(f'{j}', f"{input_methods[j]}")
        all_df['obstime'] = df.groupby('stno').resample('D')['obstime'].first().values
        all_df['stno'] = df.groupby('stno').resample('D')['stno'].first().values
        all_dff = all_df.copy()
        all_dff['obstime'] = pd.to_datetime(all_dff['obstime'], infer_datetime_format=True)
        all_dff['year'] = all_dff['obstime'].dt.year
        all_dff['month'] = all_dff['obstime'].dt.month
        all_dff['day'] = all_dff['obstime'].dt.day
        all_dff['ten_days'] = all_dff['day'].apply(lambda x:'1' if x <= 10 else ('2' if x <=20 else '3'))
        all_dff.reset_index(drop=True,inplace=True)
        all_dff2 = pd.DataFrame()
        for i in input_methods:
            ten_days_cal(all_dff2,i)
        all_dff2.fillna(-9999,inplace=True)
        all_dff2.reset_index(inplace=True)
        auto_final_df = pd.concat([auto_final_df,all_dff2])

grouped = auto_final_df.groupby('stno')
for name, group in grouped:
    group.to_csv(f'{os.getcwd()}'+'\\'+'tem'+'\\'+'auto'+'\\'+'groupby'+'\\'+f'auto_hour_station_{name}.csv',encoding='ANSI')
data_dir_merge2 = f'{os.getcwd()}'+'\\'+'tem'+'\\'+'auto'+'\\'+'groupby'
excels2 = [pd.read_csv(f'{data_dir_merge2}\\{fname2}',encoding='ANSI').reset_index(drop=True) for fname2 in os.listdir(data_dir_merge2) if os.path.splitext(fname2)[1].lower() == ".csv" ]
df_merge2 = pd.concat(excels2)
df_merge2.to_csv(f'{os.getcwd()}'+'\\'+'tem'+'\\'+'auto'+'\\'+f'auto_{year_from}_{year_to}.csv',index=False,encoding='ANSI')

 